In [1]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

In [37]:
EXP = Path("/mnt/towbin.data/shared/smarin/data/2024_02_07_lifespan_wBT318_20C")
raw_filemap = pd.read_csv(EXP / "analysis" / "report" / "analysis_filemap.csv")
raw_filemap = raw_filemap["Time Point raw".split()]
raw_filemap.head()

,Time,Point,raw
0,0,0,/mnt/towbin.data/shared/smarin/data/2024_02_07...
1,1,0,/mnt/towbin.data/shared/smarin/data/2024_02_07...
2,2,0,/mnt/towbin.data/shared/smarin/data/2024_02_07...
3,3,0,/mnt/towbin.data/shared/smarin/data/2024_02_07...
4,4,0,/mnt/towbin.data/shared/smarin/data/2024_02_07...


In [38]:
pick_n = 500

In [39]:
filemap_sample = raw_filemap.sample(pick_n, random_state=42)

In [40]:
def process_file(input_file, output_file, channel):
    img = tifffile.imread(input_file, key=channel)
    tifffile.imwrite(output_file, img)

In [41]:
Path().cwd()

PosixPath('/mnt/towbin.data/personal/bgusev/peter_work/egg_count/xgboost_training')

In [42]:
TRAINING_DATA = Path("training_data")

TRAINING_DATA_IMAGES = TRAINING_DATA / "training_images"
TRAINING_DATA_IMAGES.mkdir(parents=True, exist_ok=True)

MANUAL_SEG_IMAGES = TRAINING_DATA / "manual_seg"
MANUAL_SEG_IMAGES.mkdir(parents=True, exist_ok=True)

_RAW_IMAGES = TRAINING_DATA / "_raw"
_RAW_IMAGES.mkdir(parents=True, exist_ok=True)

In [43]:
input_files = list(filemap_sample["raw"])
output_files = [(TRAINING_DATA_IMAGES / Path(f).name).resolve() for f in input_files]

In [44]:
CHANNEL = 1
Parallel()(
    delayed(process_file)(input_file, output_file, CHANNEL)
    for input_file, output_file in tqdm(
        zip(input_files, output_files), total=len(input_files)
    )
)
print("done")

  0%|          | 0/500 [00:00<?, ?it/s]

done


In [45]:
_raw_output_files = [(_RAW_IMAGES / Path(f).name).resolve() for f in input_files]
_raw_output_files[-1]

PosixPath('/mnt/towbin.data/personal/bgusev/peter_work/egg_count/xgboost_training/training_data/_raw/Time00178_Point0017_470nm,565nm,BF_Seq37552.ome.tiff')

In [46]:
Parallel()(
    delayed(shutil.copy)(input_file, output_file)
    for input_file, output_file in tqdm(
        zip(input_files, _raw_output_files), total=len(input_files)
    )
)
print("done")

  0%|          | 0/500 [00:00<?, ?it/s]

done


In [47]:
filemap_sample["training_image"] = pd.Series(
    output_files, dtype=str, index=filemap_sample.index
)
filemap_sample["_raw_local"] = pd.Series(
    _raw_output_files, dtype=str, index=filemap_sample.index
)

# These don't exist yet, but will after finishing the manual segmentation
filemap_sample["manual_seg"] = pd.Series(
    [(MANUAL_SEG_IMAGES / Path(f).name).resolve() for f in input_files],
    dtype=str,
    index=filemap_sample.index,
)

In [53]:
filemap_sample.head()

,Time,Point,raw,training_image,_raw_local,manual_seg
21171,143,22,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
29897,265,31,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2574,666,2,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
36861,537,38,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
27774,54,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [50]:
len(filemap_sample)

500

In [52]:
filemap_sample.isna().any()

Time              False
Point             False
raw               False
training_image    False
_raw_local        False
manual_seg        False
dtype: bool

In [56]:
(
    filemap_sample.iloc[-1]["manual_seg"],
    filemap_sample.iloc[-1]["training_image"],
    filemap_sample.iloc[-1]["_raw_local"],
    filemap_sample.iloc[-1]["raw"],
)

('/mnt/towbin.data/personal/bgusev/peter_work/egg_count/xgboost_training/training_data/manual_seg/Time00178_Point0017_470nm,565nm,BF_Seq37552.ome.tiff',
 '/mnt/towbin.data/personal/bgusev/peter_work/egg_count/xgboost_training/training_data/training_images/Time00178_Point0017_470nm,565nm,BF_Seq37552.ome.tiff',
 '/mnt/towbin.data/personal/bgusev/peter_work/egg_count/xgboost_training/training_data/_raw/Time00178_Point0017_470nm,565nm,BF_Seq37552.ome.tiff',
 '/mnt/towbin.data/shared/smarin/data/2024_02_07_lifespan_wBT318_20C/raw/Time00178_Point0017_470nm,565nm,BF_Seq37552.ome.tiff')

In [54]:
filemap_sample.to_csv(TRAINING_DATA / "filemap.csv", index=False)

In [1]:
# to stop execution if jupyter's "Run All" was pressed
raise ValueError("STOP HERE")

ValueError: STOP HERE

In [2]:
# the following code is just to merge two filemaps since I manually annotated twice (100 + 500 images)

In [2]:
fmap1 = pd.read_csv(
    "/home/bgusev/new_personal/peter_work/egg_count/annot_training/filemap.csv"
)
fmap2 = pd.read_csv(
    "/home/bgusev/new_personal/peter_work/egg_count/xgboost_training/training_data/filemap.csv"
)

In [3]:
fmap1.head()

,Time,Point,raw,stardist_seg,annot_training_file,manual_seg
0,31,43,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
1,152,27,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2,171,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
3,188,18,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
4,45,28,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [4]:
fmap1 = fmap1.drop(columns="stardist_seg")
fmap1 = fmap1.rename(
    columns={
        "annot_training_file": "training_image",
    }
)

fmap1.head()

,Time,Point,raw,training_image,manual_seg
0,31,43,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
1,152,27,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2,171,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
3,188,18,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
4,45,28,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [5]:
fmap2.head()

,Time,Point,raw,training_image,_raw_local,manual_seg
0,143,22,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
1,265,31,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2,666,2,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
3,537,38,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
4,54,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [6]:
overlap = fmap1.merge(fmap2, on=["Time", "Point"], how="inner")[["Time", "Point"]]
print("Overlap: ", len(overlap))
print(f"Expected final len: {len(fmap1) + len(fmap2) - len(overlap)}")


fmap2_no_overlap = fmap2[
    ~(fmap2["Time"].isin(overlap["Time"]) & fmap2["Point"].isin(overlap["Point"]))
]
fmap2_no_overlap.head()

Overlap:  3
Expected final len: 597


,Time,Point,raw,training_image,_raw_local,manual_seg
0,143,22,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
1,265,31,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2,666,2,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
3,537,38,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
4,54,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [7]:
combined_fmap = pd.concat([fmap1, fmap2_no_overlap], ignore_index=True)
combined_fmap = combined_fmap.drop(columns="_raw_local")
print(len(combined_fmap))
combined_fmap.head()

597


,Time,Point,raw,training_image,manual_seg
0,31,43,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
1,152,27,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
2,171,29,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
3,188,18,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...
4,45,28,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/personal/bgusev/peter_work/eg...,/mnt/towbin.data/personal/bgusev/peter_work/eg...


In [8]:
combined_fmap.iloc[0]["manual_seg"]

'/mnt/towbin.data/personal/bgusev/peter_work/egg_count/annot_training/manual_seg/Time00031_Point0043_470nm,565nm,BF_Seq38278.ome.tif'

In [9]:
# I have already merged the directories, now just need to update the filemaps to point towards the new image locations

In [10]:
# Fix it, it's false because the paths still point to the old location

In [11]:
training_files = [
    str(f.resolve())
    for f in Path(
        "/home/bgusev/new_towbin/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/training_images"
    ).glob("*.ome.tiff")
]
print(len(training_files))
training_files = pd.DataFrame(training_files, columns=["training_image"])
training_files = pd.concat(
    [
        training_files,
        training_files["training_image"]
        .str.extract(r"Time(?P<Time>\d+)_Point(?P<Point>\d+)", expand=True)
        .astype(int),
    ],
    axis=1,
)
assert not training_files.isna().any().any()

manual_seg_files = [
    str(f.resolve())
    for f in Path(
        "/home/bgusev/new_towbin/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/manual_seg"
    ).glob("*ome.tif")
]
print(len(manual_seg_files))
manual_seg_files = pd.DataFrame(manual_seg_files, columns=["manual_seg"])
manual_seg_files = pd.concat(
    [
        manual_seg_files,
        manual_seg_files["manual_seg"]
        .str.extract(r"Time(?P<Time>\d+)_Point(?P<Point>\d+)", expand=True)
        .astype(int),
    ],
    axis=1,
)
assert not manual_seg_files.isna().any().any()

local_raw_files = [
    str(f.resolve())
    for f in Path(
        "/home/bgusev/new_towbin/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/_raw"
    ).glob("*.tiff")
]
print(len(local_raw_files))
local_raw_files = pd.DataFrame(local_raw_files, columns=["_raw_local"])
local_raw_files = pd.concat(
    [
        local_raw_files,
        local_raw_files["_raw_local"]
        .str.extract(r"Time(?P<Time>\d+)_Point(?P<Point>\d+)", expand=True)
        .astype(int),
    ],
    axis=1,
)
assert not local_raw_files.isna().any().any()

597
597
597


In [12]:
from functools import reduce

dfs_to_merge = [
    combined_fmap[["Time", "Point", "raw"]],
    training_files,
    manual_seg_files,
    local_raw_files,
]

full_merge = reduce(
    lambda left, right: pd.merge(left, right, on=["Time", "Point"], how="outer"),
    dfs_to_merge,
)
full_merge.head()

,Time,Point,raw,training_image,manual_seg,_raw_local
0,1,15,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...
1,1,31,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...
2,2,7,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...
3,2,42,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...
4,3,37,/mnt/towbin.data/shared/smarin/data/2024_02_07...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...,/mnt/towbin.data/shared/bgusev/stardist_egg_se...


In [13]:
full_merge.isna().any(), len(full_merge)

(Time              False
 Point             False
 raw               False
 training_image    False
 manual_seg        False
 _raw_local        False
 dtype: bool,
 597)

In [14]:
(
    full_merge.iloc[-1]["raw"],
    full_merge.iloc[-1]["training_image"],
    full_merge.iloc[-1]["manual_seg"],
    full_merge.iloc[-1]["_raw_local"],
    full_merge.iloc[-1]["Time"],
    full_merge.iloc[-1]["Point"],
)

('/mnt/towbin.data/shared/smarin/data/2024_02_07_lifespan_wBT318_20C/raw/Time00959_Point0020_470nm,565nm,BF_Seq34720.ome.tiff',
 '/mnt/towbin.data/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/training_images/Time00959_Point0020_470nm,565nm,BF_Seq34720.ome.tiff',
 '/mnt/towbin.data/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/manual_seg/Time00959_Point0020_470nm,565nm,BF_Seq34720.ome.tif',
 '/mnt/towbin.data/shared/bgusev/stardist_egg_seg/xgboost_training/training_data/_raw/Time00959_Point0020_470nm,565nm,BF_Seq34720.ome.tiff',
 959,
 20)

In [15]:
full_merge.to_csv("training_data/filemap.csv", index=False)

In [16]:
Path().cwd()

PosixPath('/mnt/towbin.data/shared/bgusev/stardist_egg_seg/xgboost_training')